In [1]:
 
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

/home/avashchilko/abbyy9sem/course_cvdl/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /home/avashchilko/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [2]:
from task1pack.utils.data import SegmentationDataset

#pip install -e .
#pip install  abbyy_course_cvdl_t2

from pathlib import Path
from course_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np
import wandb

import torch

from task1pack.utils.data import HeatmapDataset

In [3]:
#DATASET_PATH = Path() / '..' / '..' / 'midv500_compressed'
DATASET_PATH = Path() / '..' / '..' / 'data' / 'midv500_compressed'
#DATASET_PATH = Path() / '..' / '..' / '..' / '..' / '..' / '..' / 'Downloads' / 'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_ocr_t1.data.MidvPackage)

In [4]:
from torchvision.transforms import Resize, Compose, ToTensor

IMAGE_SIZE = [512, 512]

image_transforms = Compose([
    ToTensor(),
    Resize(IMAGE_SIZE),
])

target_transforms = Compose([
    Resize(IMAGE_SIZE),
])

train_dataset = SegmentationDataset(data_packs=data_packs, 
        split='train', image_transforms=image_transforms, target_transforms=target_transforms)
test_dataset = SegmentationDataset(data_packs=data_packs, 
        split='test', image_transforms=image_transforms, target_transforms=target_transforms)

print(len(train_dataset), len(test_dataset))
print(train_dataset[0][0].shape, train_dataset[0][1].shape)

10750 4250
torch.Size([3, 512, 512]) torch.Size([1, 512, 512])


In [5]:
from task1pack.utils.train import train_model, show_train_plots, train_new

In [6]:
from torch.nn import BCELoss

In [7]:
model_name = 'torch unet'

train_dataloader_config = {
    'batch_size': 8,
    'shuffle': True,
    'num_workers': 2,
}

test_dataloader_config = {
    'batch_size': 8,
    'shuffle': False,
    'num_workers': 2,
}

training_config = {
    'lr': 1e-3,
    'epochs': 100,
    'step_size': 25,
    'milestones': [25, 50, 75],
    'gamma': 0.7,
}


device = 'cuda:1'
criterion = BCELoss()

wandb.init(
    project='ocr task 1',
    name='{} {} epochs with lr={} new train'.format(model_name, training_config['epochs'], training_config['lr']),
    config={
        'train_dataloader_config': train_dataloader_config,
        'test_dataloader_config': test_dataloader_config,
        'training_config': training_config,

    "architecture": model_name,
    "dataset": "MIDV-500",
    "criterion": "BCELoss",
    "optimizer": "Adam",
    "image_size": IMAGE_SIZE,  
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vashchilkoav. Use `wandb login --relogin` to force relogin


In [8]:
train_losses, test_losses, trained_model = train_new(
    train_dataset=train_dataset, 
    test_dataset=test_dataset, 
    model=model, 
    train_dataloader_kwargs=train_dataloader_config, 
    test_dataloader_kwargs=test_dataloader_config, 
    training_kwargs=training_config,
    criterion=criterion,
    device=device,
    wandb_instance=wandb,
)

Initial val loss: 0.3497244417667389
Epoch 1:
Train loss: 0.009463250637054443
Epoch 2:
Train loss: 0.0027660082560032606
Epoch 3:
Train loss: 0.0020089864265173674
Epoch 4:
Train loss: 0.0017465533455833793
Epoch 5:
Val loss: 0.008786329999566078
Train loss: 0.0014296903973445296
Epoch 6:
Train loss: 0.0012657494517043233
Epoch 7:
Train loss: 0.0011803817469626665
Epoch 8:
Train loss: 0.0010197011288255453
Epoch 9:
Train loss: 0.0011391111183911562
Epoch 10:
Val loss: 0.003814129624515772
Train loss: 0.0011070979526266456
Epoch 11:
Train loss: 0.0007562668179161847
Epoch 12:
Train loss: 0.0008426205604337156
Epoch 13:
Train loss: 0.0007803894695825875
Epoch 14:
Train loss: 0.0007621009135618806
Epoch 15:
Val loss: 0.011337289586663246
Train loss: 0.0007635108777321875
Epoch 16:
Train loss: 0.0008116129320114851
Epoch 17:
Train loss: 0.0006858967244625092
Epoch 18:
Train loss: 0.0006572501151822507
Epoch 19:
Train loss: 0.0007351592648774385
Epoch 20:
Val loss: 0.009270232170820236
Tra

TypeError: can't convert cuda:1 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [10]:
torch.save(model.state_dict(), './unet.pth')

In [11]:
wandb.finish()

lr,██████████▅▅▅▅▅▅▅▅▅▅▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁
lr,0.00034
